# Circuit models 101

In this notebook, we will explore the basics of circuit models in AutoEIS. We will start by importing the necessary libraries.

In [1]:
import autoeis as ae
ae.visualization.set_plot_style()

## Circuit representation

In AutoEIS, circuits are represented as strings. Please refer to [circuit notation](../circuit.md) for the syntax of the circuit string. Now, let's create a sample circuit string:

In [2]:
circuit = "R1-[P2,R3]-C4-[[R5,C6],[L7,R8]]-R2"

We can visualize the circuit model using the `draw_circuit` function, which requires the `lcapy` package to be installed, and a working LaTeX installation. See [here](https://lcapy.readthedocs.io/en/latest/install.html) for more details.

In [3]:
x = ae.visualization.draw_circuit(circuit)

[11:02:24] ERROR    lcapy is not installed. Please install it using `pip install
                    lcapy`.                                                     


## Querying circuit strings

Once you have the circuit string, you can run different queries on it. We're not going to explore all of available queries here, but we'll show you a few of the most common ones. To see the full list of available queries, check out the [API reference](../modules.rst).

To get the list of components in the circuit, you can use the `get_component_labels` function:

In [4]:
ae.parser.get_component_labels(circuit)

['R1', 'P2', 'R3', 'C4', 'R5', 'C6', 'L7', 'R8', 'R2']

(The impedance of) Each component is represented by one or more parameters. To get the list of parameters that fully describe the circuit, use the `get_parameter_labels` function:

In [5]:
ae.parser.get_parameter_labels(circuit)

['R1', 'P2w', 'P2n', 'R3', 'C4', 'R5', 'C6', 'L7', 'R8', 'R2']

Note that components and parameters are not the same, despite the fact that for single-parameter components they're represented by the same string. For instance, `R1` is both a parameter and a component, but `P2` is a component, which is described by the parameters `P2w` and `P2n`.

You can also validate the circuit using the `validate_circuit` function in case you're not sure if the circuit is valid:

In [6]:
ae.parser.validate_circuit(circuit)

True

Let's try to validate an invalid circuit string:

In [7]:
ae.parser.validate_circuit("R1-[R2,P3]-R1")

AssertionError: Duplicate elements found: {'R1'}